 PCA

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [ ]:
# Caricare il dataset di anomaly detection
#dataset = pd.read_csv('../../KDDTrain+.txt')
dataset = pd.read_csv('../DATA/nsl-kdd/KDDTrain+.txt')
dataset.head()

In [ ]:
columns = (['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent','hot'
,'num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root','num_file_creations'
,'num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count','srv_count','serror_rate'
,'srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count','dst_host_srv_count'
,'dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate'
,'dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','outcome','level'])
dataset.columns = columns
dataset.loc[dataset['outcome'] == "normal", "outcome"] = 'normal'
dataset.loc[dataset['outcome'] != 'normal', "outcome"] = 'attack'
dataset.head()

In [ ]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.decomposition import PCA

# Colonne categoriche da escludere dal RobustScaler
cat_cols = ['is_host_login','protocol_type','service','flag','land',
            'logged_in','is_guest_login', 'level', 'outcome']

# Preprocessing numerico + encoding categoriale
def preprocess(df_train, df_test):
    # Separazione numeriche
    num_train = df_train.drop(cat_cols, axis=1)
    num_test = df_test.drop(cat_cols, axis=1)
    num_cols = num_train.columns

    # Scaling robusto solo sul training set
    scaler = RobustScaler()
    scaled_train = scaler.fit_transform(num_train)
    scaled_test = scaler.transform(num_test)

    df_train_scaled = pd.DataFrame(scaled_train, columns=num_cols, index=df_train.index)
    df_test_scaled = pd.DataFrame(scaled_test, columns=num_cols, index=df_test.index)

    # Ricostruzione dataframe
    df_train_final = df_train.drop(num_cols, axis=1).copy()
    df_test_final = df_test.drop(num_cols, axis=1).copy()

    df_train_final[num_cols] = df_train_scaled
    df_test_final[num_cols] = df_test_scaled

    # One-hot encoding delle categoriche
    df_train_final = pd.get_dummies(df_train_final, columns=['protocol_type', 'service', 'flag'])
    df_test_final = pd.get_dummies(df_test_final, columns=['protocol_type', 'service', 'flag'])

    # Allineamento colonne dopo one-hot (test potrebbe non avere tutte le categorie)
    df_test_final = df_test_final.reindex(columns=df_train_final.columns, fill_value=0)

    return df_train_final, df_test_final


In [ ]:
# Split prima del preprocessing
train_df, test_df = train_test_split(dataset, test_size=0.2, random_state=42, stratify=dataset['outcome'])

# Preprocessing numerico e categorico
train_processed, test_processed = preprocess(train_df.copy(), test_df.copy())

# Split X/y
X_train = train_processed.drop(columns=['outcome']).values
y_train = train_processed['outcome'].values

X_test = test_processed.drop(columns=['outcome']).values
y_test = test_processed['outcome'].values

# Scaling MinMax (post-preprocessing, pre-PCA)
minmax_scaler = MinMaxScaler()
X_train_scaled = minmax_scaler.fit_transform(X_train)
X_test_scaled = minmax_scaler.transform(X_test)

# PCA solo su training
n=8
n_components = n
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# (opzionale) Riscalatura anche dopo PCA
X_train_pca = minmax_scaler.fit_transform(X_train_pca)
X_test_pca = minmax_scaler.transform(X_test_pca)

# Variance ratio (quanto "spiega" ogni componente)
print("Explained variance ratio:", pca.explained_variance_ratio_)

# Se vuoi convertire in dataframe:
#train_pca_df = pd.DataFrame(X_train_pca, columns=[f'PC{i+1}' for i in range(n_components)])
#train_pca_df['Outcome'] = y_train

#test_pca_df = pd.DataFrame(X_test_pca, columns=[f'PC{i+1}' for i in range(n_components)])
#test_pca_df['Outcome'] = y_test


In [ ]:
# Calcola il 5% dei dati disponibili per ciascuna classe nel training set
n_train = int(len(X_train_pca[y_train == 'normal']) * 0.02)
#n_attack_train = int(len(X_train_pca[y_train == 'attack']) * 0.05)

# Calcola il 5% dei dati disponibili per la classe 'normal' nel test set
n_test = int(len(X_test_pca[y_test == 'normal']) * 0.02)

# Seleziona i dati per il training
benign_data_T = X_train_pca[y_train == 'normal'][:n_train]

# Seleziona i dati per il test
benign_data_Ttest = X_test_pca[y_test == 'normal'][:n_test]
outlier_dataT = X_test_pca[y_test == 'attack'][:n_test]

# Stampa le forme dei dataset selezionati
print("Benigni per training:", benign_data_T.shape)
print("Attacchi per test:", outlier_dataT.shape)
print("Benigni per test:", benign_data_Ttest.shape)

# Visualizza il primo esempio dei benigni per training
print("Primo benigno per training:", benign_data_T[0])
print("Primo benigno per test:", benign_data_Ttest[0])
print("Primo maligno per test:", outlier_dataT[0])


In [ ]:
# Library imports
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pennylane as qml

# Pytorch imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from scipy.stats import ks_2samp

# random seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [ ]:


# creazione di un DataLoader con dati normali
dataloader = DataLoader(benign_data_T, batch_size=128, shuffle=True)
all_batches = []
# iterare sui batch
for batch_features in dataloader:
    print("Batch features shape:", batch_features.shape)  # Es. torch.Size([32, 4])
    all_batches.append(batch_features)

all_real_data = torch.cat(all_batches, dim=0)
print(all_real_data.shape)
#print(benign_data_T)


In [ ]:
# creazione di un DataLoader con dati di test reali
dataloaderTestReal = DataLoader(benign_data_Ttest, batch_size=128, shuffle=True)
all_batches = []
# iterare sui batch
for batch_features in dataloaderTestReal:
    print("Batch features shape:", batch_features.shape)  # Es. torch.Size([32, 4])
    all_batches.append(batch_features)

all_real_dataTest = torch.cat(all_batches, dim=0)
print(all_real_dataTest.shape)

# creazione di un DataLoader con dati di test di attacco
dataloaderTestOut = DataLoader(outlier_dataT, batch_size=128, shuffle=True)
all_batches = []
# iterare sui batch
for batch_features in dataloaderTestOut:
    print("Batch features shape:", batch_features.shape)  # Es. torch.Size([32, 4])
    all_batches.append(batch_features)

all_out_dataTest = torch.cat(all_batches, dim=0)
print(all_out_dataTest.shape)

In [ ]:


class Discriminator(nn.Module):
    """Discriminatore completamente connesso per dati numerici"""

    def __init__(self, input_size):
        """
        Args:
            input_size (int): Numero di caratteristiche (feature) in input.
        """
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            # Strato di input -> primo hidden layer
            nn.Linear(input_size, 64),
            nn.ReLU(),
            # Primo hidden layer -> secondo hidden layer
            nn.Linear(64, 16),
            nn.ReLU(),
            # Secondo hidden layer -> output (probabilità)
            nn.Linear(16, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.model(x)

In [ ]:

# Parametri del generatore
n_qubits = n  # Numero di qubit
q_depth = 4  # Profondità del circuito quantistico

# Quantum simulator
dev = qml.device("lightning.qubit", wires=n_qubits)
# Enable CUDA device if available
device = torch.device("cpu")
print(device)


#feature map
def basic_feature_map(x):
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
    for i in range(n_qubits):
        qml.RZ(x[i], wires=i)
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
        qml.RZ((x[i] * x[i + 1]), wires=i + 1)
        
def basic_feature_map1(x):
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
    for i in range(n_qubits):
        qml.RZ(x[i], wires=i)
        qml.RY(x[i], wires=i)
    # Rimuovo le rotazioni non lineari eccessive
    # oppure aggiungo entanglement leggero:
    
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])


def basic_feature_map2(x):
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
    for i in range(n_qubits):
        qml.RY(x[i], wires=i)
    # Entanglement ciclico leggero
    for i in range(n_qubits):
        qml.CZ(wires=[i, (i+1) % n_qubits])


        
# === ANSATZ EfficientSU2 ===
def efficient_su2(weights, wires):
    num_qubits = len(wires)
    depth = weights.shape[0]

    for layer in range(depth - 1):
        for i in range(num_qubits):
            qml.RY(weights[layer, i, 0], wires=wires[i])
            qml.RZ(weights[layer, i, 1], wires=wires[i])

        # Entanglement lineare tra i qubit
        for i in range(num_qubits - 1):
            qml.CNOT(wires=[wires[i], wires[i + 1]])
        qml.CNOT(wires=[wires[-1], wires[0]])

    # Blocco finale di rotazioni
    for i in range(num_qubits):
        qml.RY(weights[-1, i, 0], wires=wires[i])
        qml.RZ(weights[-1, i, 1], wires=wires[i])


def two_local_ansatz(weights, wires):
    num_qubits = len(wires)
    depth = weights.shape[0]

    for layer in range(depth):
        # Rotazioni su ogni qubit
        for i in range(num_qubits):
            qml.RY(weights[layer, i, 0], wires=wires[i])
            qml.RZ(weights[layer, i, 1], wires=wires[i])

        # Entanglement lineare (a catena)
        for i in range(num_qubits - 1):
            qml.CZ(wires=[wires[i], wires[i + 1]])

@qml.qnode(dev, interface="torch", diff_method="parameter-shift")
def quantum_circuit(latent_vector, weights):
    # Codifica del rumore latente (opzionale: feature map)
    for i in range(n_qubits):
        qml.RY(latent_vector[i], wires=i)

    # Ansatz stile TwoLocal
    two_local_ansatz(weights, wires=range(n_qubits))

    return qml.probs(wires=range(n_qubits))




In [ ]:
def partial_measure(noise, weights):

    probs = quantum_circuit(noise, weights)  # Lista di 2^n valori di probabilità
    num_states = 2 ** n_qubits

    transformed_data = probs[:n_qubits].clone()
    for i in range(n_qubits):
        # Somma i valori alternati dalla riga corrente (step di i+1)
        sum_value = 0
        # Ciclo per sommare in blocchi alternati di i+1
        for start in range(0, num_states,  2**(i + 1)):  # Ogni passo è di 2*(i+1)
            sum_value += sum(probs[start:start + 2**(i)])  # Sommiamo i primi i+1 element  
        transformed_data[i] = sum_value
    return transformed_data


In [ ]:
class QuantumGenerator(nn.Module):
    """Generatore quantistico per dati numerici."""

    def __init__(self, latent_dim):
        """
        Args:
            latent_dim (int): Dimensione del vettore latente.
        """
        super(QuantumGenerator, self).__init__()
        self.latent_dim = latent_dim
        # Inizializza i parametri del circuito quantistico
        self.q_params = nn.Parameter(0.1 * ((torch.rand(q_depth, n_qubits,2) - 0.5) * torch.pi), requires_grad=True)
        

    def forward(self, batch_size):

        epsilon = 0.5
        generated_data = []
        for _ in range(batch_size):
            # Campiona un vettore latente casuale
            #latent_vector = torch.full((self.latent_dim,), torch.pi ) * (torch.rand(self.latent_dim) - 0.5)   #torch.rand(self.latent_dim) * (torch.pi)
            #latent_vector = torch.rand(self.latent_dim)
            latent_vector = torch.rand(latent_dim, device=device) * (math.pi)
            
            #latent_vector = (torch.rand(self.latent_dim) - 0.5) * np.pi / 2  # range [-π/4, π/4]
            
            # Ottieni l'output del circuito quantistico
            probs = partial_measure(latent_vector, self.q_params)

             # Convert probs to float32
            probs = probs.type(torch.float32)
            generated_data.append(probs)


        generated_data = torch.stack(generated_data, dim=0)
        return generated_data.squeeze()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.linalg import sqrtm


# Funzione per confrontare le distribuzioni
def compare_distributions(real_data, fake_data, epoch):
    real_data = real_data.cpu().numpy()
    # Se fake_data è un tensore PyTorch, convertilo in NumPy
    if isinstance(fake_data, torch.Tensor):
        fake_data = fake_data.cpu().numpy()
    print(real_data.shape)
    print(fake_data.shape)
    print(fake_data)
    num_features = real_data.shape[1]

    # Crea un subplot per ogni feature
    fig, axes = plt.subplots(1, num_features, figsize=(5 * num_features, 4))
    if num_features == 1:
        axes = [axes]  # Assicura che funzioni anche con una singola feature

    for i in range(num_features):
        sns.kdeplot(real_data[:, i], label="Real", ax=axes[i], color="blue")
        sns.kdeplot(fake_data[:, i], label="Generated", ax=axes[i], color="orange")
        axes[i].set_title(f"Feature {i+1}")
        axes[i].legend()

    plt.suptitle(f"Epoch {epoch} - Distribution Comparison")
    plt.tight_layout()
    plt.show()

def plot_probabilities(probs1):
    """
    Genera un grafico a barre per visualizzare le 16 probabilità di uscita del circuito quantistico.

    Args:
        probs
    """
    # Ottieni le probabilità dal circuito quantistico
    probs=probs1.detach().numpy()

    # Indici degli stati (da 0000 a 1111 in binario)
    states = [format(i, '04b') for i in range(len(probs))]

    # Creazione del grafico
    plt.figure(figsize=(10, 5))
    plt.bar(states, probs, color='royalblue', alpha=0.7)

    # Etichette e titolo
    plt.xlabel("Stati Quantistici (10 Qubit)")
    plt.ylabel("Probabilità")
    plt.title("Distribuzione delle Probabilità del Circuito Quantistico")
    plt.ylim(0, 1)  # Imposta il limite massimo a 1
    plt.grid(axis="y", linestyle="--", alpha=0.6)

    # Mostra i valori sopra le barre
    for i, p in enumerate(probs):
        plt.text(i, p + 0.02, f"{p:.2f}", ha='center', fontsize=10)

    plt.show()


def plot_losses(generator_losses, discriminator_losses):
    plt.figure(figsize=(10, 5))
    plt.plot(generator_losses, label="Generator Loss", color='blue')
    plt.plot(discriminator_losses, label="Discriminator Loss", color='red')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Generator and Discriminator Loss During Training")
    plt.legend()
    plt.grid()
    plt.show()


def compute_discriminator_metrics(discriminator, dataloadreal, dataloaderout, device):
    y_true, y_pred = [], []
    y_true2, y_pred2 = [], []
    with torch.no_grad():
        for real_data in dataloadreal:
            real_data = real_data.to(device).float()
            batch_size = real_data.size(0)

            real_labels = torch.ones(batch_size, device=device)
            #fake_data = generator(batch_size).to(device)
            #fake_labels = torch.zeros(batch_size, device=device)

            res= discriminator(real_data).view(-1)
            real_preds = res > 0.5

            #print(f"real_preds; {res}")

            y_true.extend(real_labels.cpu().numpy())
            y_pred.extend(real_preds.cpu().numpy())

        for out_data in dataloaderout:
            out_data = out_data.to(device).float()
            batch_size = out_data.size(0)

            fake_labels = torch.zeros(batch_size, device=device)

            res= discriminator(out_data).view(-1)
            fake_preds = res > 0.5

            #print(f"fake_preds; {res}")

            y_true2.extend(fake_labels.cpu().numpy())
            y_pred2.extend(fake_preds.cpu().numpy())

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    accuracy2 = accuracy_score(y_true2, y_pred2)
    precision2 = precision_score(y_true2, y_pred2)
    recall2 = recall_score(y_true2, y_pred2)
    f12 = f1_score(y_true2, y_pred2)


    return accuracy, precision, recall, f1, accuracy2, precision2, recall2, f12

def compute_generator_metrics(discriminator, generator, batch_size, device):
    with torch.no_grad():
        fake_data = generator(batch_size).to(device)
        fake_preds = discriminator(fake_data).view(-1) > 0.5

        y_true = torch.ones(batch_size, device=device).cpu().numpy()
        y_pred = fake_preds.cpu().numpy()

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    return accuracy, precision, recall, f1

def calculate_fid(real_features, generated_features):
    # Calcola la media e la covarianza per entrambi i dataset
    mu_real, sigma_real = np.mean(real_features, axis=0), np.cov(real_features, rowvar=False)
    mu_gen, sigma_gen = np.mean(generated_features, axis=0), np.cov(generated_features, rowvar=False)

    # Calcola la distanza di Frechet
    fid = np.sum((mu_real - mu_gen) ** 2) + np.trace(sigma_real + sigma_gen - 2 * sqrtm(sigma_real @ sigma_gen))
    
    # A volte sqrtm restituisce valori complessi, quindi prendiamo solo la parte reale
    if np.iscomplexobj(fid):
        fid = fid.real

    return fid

def plot_fid(epochs_recorded, fid_scores):
        # Disegna il grafico FID
        plt.figure(figsize=(8, 5))
        plt.plot(epochs_recorded, fid_scores, marker='o', linestyle='-', color='b', label="FID Score")
        plt.xlabel("Epoch")
        plt.ylabel("FID Score")
        plt.title("FID Score over Epochs")
        plt.legend()
        plt.grid(True)
        plt.show()


def plot_2fid(epochs_recorded, fid_scores, genFid):
  
    # Disegna il grafico FID per entrambi i valori
    plt.figure(figsize=(8, 5))
    
    # Plotta il FID originale
    plt.plot(epochs_recorded, fid_scores, marker='o', linestyle='-', color='b', label="FID Score")

    # Plotta il secondo FID (genFid)
    plt.plot(epochs_recorded, genFid, marker='s', linestyle='--', color='r', label="Generated FID")

    # Etichette e titolo
    plt.xlabel("Epoch")
    plt.ylabel("FID Score")
    plt.title("FID Score over Epochs")

    # Aggiunge la legenda
    plt.legend()

    # Mostra la griglia
    plt.grid(True)

    # Mostra il grafico
    plt.show()


def plot_accuracy(epochs, d_accuracies, g_accuracies,d_accuracy2):
    """
    Plotta l'accuracy del Discriminatore e del Generatore nel tempo.

    Parametri:
    - epochs: lista delle epoche
    - d_accuracies: lista dei valori di accuracy del discriminatore
    - g_accuracies: lista dei valori di accuracy del generatore
    """
    plt.figure(figsize=(8, 5))
    plt.plot(epochs, d_accuracies, label="Discriminatore Reali", marker='o', linestyle='-')
    plt.plot(epochs, g_accuracies, label="Generatore", marker='s', linestyle='--')
    plt.plot(epochs, d_accuracies2, label="Discriminatore Attacchi", marker='o', linestyle='-')

    plt.xlabel("Epoche")
    plt.ylabel("Accuracy")
    plt.title("Andamento dell'Accuracy nel tempo")
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
# Inizializza il generatore
latent_dim = n_qubits  # Dimensione del vettore latente
generator = QuantumGenerator(latent_dim=latent_dim)

# Genera un batch di dati
batch_size = len(all_real_data)

# generate the data
generated_data = generator(batch_size=batch_size)

# Convert the generated_data to float32 before using it.
generated_data = generated_data.type(torch.float32)

torch.set_printoptions(precision=3, sci_mode=False)
print("Dati generati:")
#print(generated_data)
print(generated_data.shape)
with torch.no_grad():
    compare_distributions(all_real_data, generated_data , 0)





In [ ]:
print(generated_data[500])

In [ ]:

# Iperparametri più stabili
lrd = 0.0005  # learning rate
lrg = 0.005  # learning rate
b1 = 0.7  # first momentum parameter
b2 = 0.999  # second momentum parameter


num_iter = 500
#batch_size = 128 ... modificare in dataloader


# Numero di feature
input_size = n

# Inizializzazioni
discriminator = Discriminator(input_size=input_size).to(device)
generator = QuantumGenerator(latent_dim=n_qubits).to(device)

#modalita addestramento
generator.train()
discriminator.train()





# Ottimizzatori più stabili beta1 0.5 meno inerzia, beta2 0.999 velocita aggiornamento pesi
#optD = optim.Adam(discriminator.parameters(), lr=lrD, betas=(0.5, 0.999))
#optG = optim.Adam(generator.parameters(), lr=lrG, betas=(0.5, 0.999))
optG = optim.Adam(generator.parameters(), lr=lrg, betas=(b1, b2), weight_decay=0.005) # to avoid overfitting
optD = optim.Adam(discriminator.parameters(), lr=lrd, betas=(b1, b2), weight_decay=0.005)

#optD = optim.SGD(discriminator.parameters(), lr=lrD)
#optG = optim.SGD(generator.parameters(), lr=lrG)




# loss
criterion = torch.nn.BCELoss()

#raccolta tutti i dati reali
#all_real_data = torch.cat([data[0] for data in dataloader], dim=0).to(device).float()

generator_losses = []
discriminator_losses = []
fid_scores = []
epochs_recorded = []
genfid_scores = []
d_accuracies = []  # Lista delle accuracy del discriminatore
d_accuracies2 = []
g_accuracies = []

# Training loop
for epoch in range(num_iter):
    total_d_loss = 0
    total_g_loss = 0

    for i, real_data in enumerate(dataloader):
        real_data = real_data.to(device)
        current_batch_size = real_data.size(0)
        #128 modifica in dataloader

        # da double a float32
        real_data = real_data.float()
        
        # Azzera i gradienti
        optD.zero_grad()

        # Dati reali
        real_output = discriminator(real_data).view(-1)
        real_labels = torch.ones(current_batch_size, 1, device=device, dtype=torch.float32).view(-1)
        real_loss = criterion(real_output, real_labels)

        # Genera dati falsi
        #noise = torch.randn(current_batch_size, 4, device=device) //noise generato in funzione quantumgenerator
        fake_data = generator(current_batch_size).detach()

        # Dati falsi
        fake_labels = torch.zeros(current_batch_size, 1, device=device, dtype=torch.float32).view(-1)
        fake_output = discriminator(fake_data).view(-1)
        fake_loss = criterion(fake_output, fake_labels)

        # Perdita discriminatore
        real_loss.backward()
        fake_loss.backward()
        d_loss = real_loss + fake_loss
        optD.step()

        # Addestramento generatore //senza .detach
        optG.zero_grad()
        fake_data = generator(current_batch_size)
        generator_output = discriminator(fake_data).view(-1)

        # Il generatore cerca di ingannare il discriminatore crietrion/obbeittivo: dati generati classsficiati come real 1
        g_labels = torch.ones(current_batch_size, 1, device=device, dtype=torch.float32).view(-1)
        g_loss = criterion(generator_output, g_labels)

        #print("Gradients before loss.backward():", generator.q_params.grad)
        g_loss.backward()

        #print("Gradients after loss.backward():", generator.q_params.grad)


        optG.step()
        #print("Gradients after optimizer step:", generator.q_params.grad)


        # Accumula le perdite
        total_d_loss += d_loss.item()
        total_g_loss += g_loss.item()

    discriminator_losses.append(total_d_loss / (i + 1))
    generator_losses.append(total_g_loss / (i + 1))


    # Stampa medie delle perdite
    if 1:
        print(f"Epoch {epoch}/{num_iter}, "
              f"Avg Loss D: {total_d_loss/(i+1):.4f}, "
              f"Avg Loss G: {total_g_loss/(i+1):.4f}")
            # Plot delle distribuzioni ad ogni epoca
        
    if epoch % 1 == 0: #mostra plot ogni 50epoch
      with torch.no_grad():
        fake_data_sample = generator(len(all_real_data))  # Genera lo stesso numero di allrealdata
        compare_distributions(all_real_data, fake_data_sample, epoch)
        fid_score = calculate_fid(all_real_data.cpu().numpy(), fake_data_sample.cpu().numpy())
        print(f"FID Score: {fid_score}")
        

        random_data = np.random.rand(len(all_real_data), n_qubits)
        genFid = calculate_fid(all_real_data.cpu().numpy(), random_data)
        print(f"genFID score; {genFid}")
        #compare_distributions(all_real_data, random_data, epoch)

        # Salva il valore di FID e l'epoca corrispondente
        fid_scores.append(fid_score)
        genfid_scores.append(genFid)
        epochs_recorded.append(epoch)
        plot_2fid(epochs_recorded,fid_scores,genfid_scores)
        plot_fid(epochs_recorded,fid_scores)
          
        #ks_stat, p_value = ks_2samp(all_real_data, fake_data_sample)
        #print(f"KS Statistic: {ks_stat}")
        #print(f"P-value: {p_value}")
        plot_losses(generator_losses, discriminator_losses)

        d_accuracy, d_precision, d_recall, d_f1, d_accuracy2, d_precision2, d_recall2, d_f12 = compute_discriminator_metrics(discriminator, dataloaderTestReal, dataloaderTestOut, device)
        g_accuracy, g_precision, g_recall, g_f1 = compute_generator_metrics(discriminator, generator, len(dataloader.dataset), device)

        print(f"Metrics at epoch {epoch}:")
        print(f"  Discriminator real- Accuracy={d_accuracy:.4f}, Precision={d_precision:.4f}, Recall={d_recall:.4f}, F1-score={d_f1:.4f}")
        print(f"  Discriminator fake- Accuracy={d_accuracy2:.4f}, Precision={d_precision2:.4f}, Recall={d_recall2:.4f}, F1-score={d_f12:.4f}")
        print(f"  Generator - Accuracy={g_accuracy:.4f}, Precision={g_precision:.4f}, Recall={g_recall:.4f}, F1-score={g_f1:.4f}")
        d_accuracies.append(d_accuracy)
        d_accuracies2.append(d_accuracy2)  
        g_accuracies.append(g_accuracy)
        plot_accuracy(epochs_recorded, d_accuracies, g_accuracies,d_accuracy2)



In [ ]:
# Usa argsort per ordinare gli indici in base ai valori crescenti
min_indices = np.argsort(fid_scores)[:50]
mean_gen = np.mean(genfid_scores)

print(f"media fid generato {mean_gen}")
# Stampa posizione e valore corrispondente
print("Primi 10 minimi (posizione -> valore):")
for idx in min_indices:
    print(f"Posizione: {idx}, Valore: {fid_scores[idx]}")


# Definisci la dimensione dell'intervallo
bin_size = 50

# Trova il valore massimo per sapere quanti intervalli servono
max_index = max(min_indices)
num_bins = (max_index // bin_size) + 1  # numero di intervalli necessari


# Inizializza contenitori
counts = [0] * num_bins
values_per_bin = [[] for _ in range(num_bins)]

# Raccoglie valori per ogni intervallo
for idx in min_indices:
    bin_index = idx // bin_size
    counts[bin_index] += 1
    values_per_bin[bin_index].append(fid_scores[idx])

# Stampa risultati con media
for i in range(num_bins):
    start = i * bin_size
    end = start + bin_size - 1
    count = counts[i]
    valori = values_per_bin[i]
    media = np.mean(valori) if valori else 0
    print(f"Intervallo {start}-{end}: {count} indice/i, media valori = {media:.4f}")

In [ ]:
#capire se riducendo i dati prima di fare pca cambia il risultato

In [ ]:
#capire come fa a partire da un accuracy cosi alta

In [ ]:
#migliorare il generatore e cambiare i LR